In [1]:
import numpy as np
import pandas as pd
from factor_test import N_tile
import matplotlib.pyplot as plt
import sys
from scipy.stats import randint
import matplotlib.pyplot as plt 
import seaborn as sns  
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
ticker='BTCUSDT'
#%%read data
data_train=pd.read_csv('Training_factors_label_V1.csv')
data_train.time_index=pd.to_datetime(data_train.time_index)
data_train.set_index('time_index',inplace=True)
data_train=data_train.iloc[:,1:]

data_vali=pd.read_csv('Validation_factors_label_V1.csv')
data_vali.time_index=pd.to_datetime(data_vali.time_index)
data_vali.set_index('time_index',inplace=True)
data_vali=data_vali.iloc[:,1:]

In [3]:
#define lookback
start_date = data_train.index[0]
start_ind = np.where(data_train.index == start_date)[0][0]
lag_period = 30

def create_lookback_data(df,start_ind, lag_period,y_index,forward_period, dropnan=True):
    cols, names = list(), list()
    columns = df.columns
    n_vars = len(columns)
    for i in range(lag_period,-1, -1):
        cols.append(df.shift(i))
        if i == 0:
            names += [('%s(t)' % (columns[j])) for j in range(n_vars)]
        else:
            names += [('%s(t-%d)' % (columns[j], i)) for j in range(n_vars)]
    col_y = columns[y_index]
    for i in range(1, forward_period+1):
        cols.append(df.iloc[:,y_index].shift(-i))
        names += [('%s(t+%d)' % (col_y, i))]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg.iloc[start_ind:,:]

In [4]:
#create lookback dataset
y_index = 0
data_train = create_lookback_data(data_train,start_ind,30,y_index,7,dropnan=False)
data_train.head()

,label(t-30),regular_Ceiling_Floors_15(t-30),regular_Trend_Speed_3(t-30),regular_Last2_trend(t-30),irregular_Dynamic_Pressure(t-30),irregular_liquid_orderbook_slope(t-30),regular_Consecutive_Large_Order_Flow(t-30),regular_Ceiling_Floors_9(t-30),regular_Ceiling_Floors_17(t-30),regular_Ceiling_Floors_13(t-30),...,regular_Ceiling_Floors_19(t),regular_Ceiling_Floors_7(t),irregular_liquid_hour(t),label(t+1),label(t+2),label(t+3),label(t+4),label(t+5),label(t+6),label(t+7)
time_index,,,,,,,,,,,,,,,,,,,,,
2019-01-08 20:27:15.340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,0.000529,0.000527,0.000525,0.000524,0.000524,0.000523,0.000522
2019-01-08 20:27:15.380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,0.000527,0.000525,0.000524,0.000524,0.000523,0.000522,0.000512
2019-01-08 20:27:15.747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,0.000525,0.000524,0.000524,0.000523,0.000522,0.000512,0.000509
2019-01-08 20:27:16.073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,0.000524,0.000524,0.000523,0.000522,0.000512,0.000509,0.000509
2019-01-08 20:27:16.133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.0,0.000524,0.000523,0.000522,0.000512,0.000509,0.000509,0.000508


In [5]:
start_date = data_vali.index[0]
start_ind = np.where(data_vali.index == start_date)[0][0]
lag_period = 30
y_index = 0
data_vali = create_lookback_data(data_vali,start_ind,30,y_index,7,dropnan=False)
data_vali.tail()

,label(t-30),regular_Ceiling_Floors_15(t-30),regular_Trend_Speed_3(t-30),regular_Last2_trend(t-30),irregular_Dynamic_Pressure(t-30),irregular_liquid_orderbook_slope(t-30),regular_Consecutive_Large_Order_Flow(t-30),regular_Ceiling_Floors_9(t-30),regular_Ceiling_Floors_17(t-30),regular_Ceiling_Floors_13(t-30),...,regular_Ceiling_Floors_19(t),regular_Ceiling_Floors_7(t),irregular_liquid_hour(t),label(t+1),label(t+2),label(t+3),label(t+4),label(t+5),label(t+6),label(t+7)
time_index,,,,,,,,,,,,,,,,,,,,,
2019-02-06 15:09:10.163,0.001612,0.001646,-5.079475e-07,-0.001214,0.553061,80221.452144,-0.000749,0.001148,0.001652,0.000978,...,-0.000825,0.000682,15.0,0.001628,0.001629,0.001629,0.001629,NaN,NaN,NaN
2019-02-06 15:09:11.123,0.001615,0.001646,-5.079475e-07,-0.001214,0.460543,85267.339842,-0.000749,0.001148,0.001652,0.000978,...,-0.000825,0.000682,15.0,0.001629,0.001629,0.001629,NaN,NaN,NaN,NaN
2019-02-06 15:09:12.000,0.001617,0.001646,-5.079475e-07,-0.001214,0.417240,87629.082210,-0.000749,0.001148,0.001652,0.000978,...,-0.000825,0.000682,15.0,0.001629,0.001629,NaN,NaN,NaN,NaN,NaN
2019-02-06 15:09:12.120,0.001617,0.001646,-5.079475e-07,-0.001214,0.417015,87641.335810,-0.000749,0.001148,0.001652,0.000978,...,-0.000825,0.000682,15.0,0.001629,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-06 15:09:12.127,0.001619,0.001646,-5.079475e-07,-0.001214,0.366163,90414.765604,-0.000749,0.001148,0.001652,0.000978,...,-0.000825,0.000682,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_train_y=data_train['label(t)']
data_train_x=data_train.copy()
del data_train_x['label(t)']

data_vali_y=data_vali['label(t)']
data_vali_x=data_vali.copy()
del data_vali_x['label(t)']

print(data_train_y.shape,data_train_x.shape,data_vali_y.shape,data_vali_x.shape)

(1000000,) (1000000, 1494) (1000000,) (1000000, 1494)


In [7]:
data_train_x.fillna(0,inplace=True)
data_train_x.replace(np.inf,0,inplace=True)
data_train_x.replace(-np.inf,0,inplace=True)

data_train_y.fillna(0,inplace=True)
data_train_y.replace(np.inf,0,inplace=True)
data_train_y.replace(-np.inf,0,inplace=True)

data_vali_x.fillna(0,inplace=True)
data_vali_x.replace(np.inf,0,inplace=True)
data_vali_x.replace(-np.inf,0,inplace=True)

data_vali_y.fillna(0,inplace=True)
data_vali_y.replace(np.inf,0,inplace=True)
data_vali_y.replace(-np.inf,0,inplace=True)

In [8]:
train_X = data_train_x.values.reshape((data_train_x.shape[0], 1, data_train_x.shape[1]))
train_Y = data_train_y.values
val_X = data_vali_x.values.reshape((data_vali_x.shape[0], 1, data_vali_x.shape[1]))
val_Y = data_vali_y.values
print(train_X.shape,train_Y.shape,val_X.shape,val_Y.shape)

(1000000, 1, 1494) (1000000,) (1000000, 1, 1494) (1000000,)


In [9]:
#stacked LSTMs
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit model
history = model.fit(train_X, train_Y, epochs=10, batch_size=100, validation_data=(val_X, val_Y), verbose=1, shuffle=False)

W0301 22:44:34.685351 4556410304 deprecation_wrapper.py:119] From /Users/Chloe/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 1000000 samples, validate on 1000000 samples
Epoch 1/10
1000000/1000000 [==============================] - 153s 153us/step - loss: 3.3561e-05 - val_loss: 2.6547e-06
Epoch 2/10
1000000/1000000 [==============================] - 162s 162us/step - loss: 4.6804e-06 - val_loss: 2.5606e-06
Epoch 3/10
1000000/1000000 [==============================] - 175s 175us/step - loss: 4.7755e-06 - val_loss: 2.5592e-06
Epoch 4/10
1000000/1000000 [==============================] - 166s 166us/step - loss: 4.7717e-06 - val_loss: 2.5615e-06
Epoch 5/10
1000000/1000000 [==============================] - 183s 183us/step - loss: 4.7714e-06 - val_loss: 2.5629e-06
Epoch 6/10
1000000/1000000 [==============================] - 371s 371us/step - loss: 4.7706e-06 - val_loss: 2.5623e-06
Epoch 7/10
1000000/1000000 [==============================] - 278s 278us/step - loss: 4.7704e-06 - val_loss: 2.5643e-06
Epoch 8/10
1000000/1000000 [==============================] - 153s 153us/step - loss: 4.7735e-06 - val_los

In [11]:
#%%test data
Test1=pd.read_csv('Test1_factors_label_V1.csv')
Test1.time_index=pd.to_datetime(Test1.time_index)
Test1.set_index('time_index',inplace=True)
Test1=Test1.iloc[:,1:]

Test2=pd.read_csv('Test2_factors_label_V1.csv')
Test2.time_index=pd.to_datetime(Test2.time_index)
Test2.set_index('time_index',inplace=True)
Test2=Test2.iloc[:,1:]

In [12]:
start_date = Test1.index[0]
start_ind = np.where(Test1.index == start_date)[0][0]
lag_period = 30
y_index = 0
Test1 = create_lookback_data(Test1,start_ind,30,y_index,7,dropnan=False)

start_date = Test2.index[0]
start_ind = np.where(Test2.index == start_date)[0][0]
lag_period = 30
y_index = 0
Test2 = create_lookback_data(Test2,start_ind,30,y_index,7,dropnan=False)

In [14]:
Test1_y=Test1['label(t)']
Test1_x=Test1.copy()
del Test1_x['label(t)']

Test2_y=Test2['label(t)']
Test2_x=Test2.copy()
del Test2_x['label(t)']

print(Test1_y.shape,Test1_x.shape,Test2_y.shape,Test2_x.shape)

(1000000,) (1000000, 1494) (1000000,) (1000000, 1494)


In [15]:
Test1_x.fillna(0,inplace=True)
Test1_x.replace(np.inf,0,inplace=True)
Test1_x.replace(-np.inf,0,inplace=True)

Test1_y.fillna(0,inplace=True)
Test1_y.replace(np.inf,0,inplace=True)
Test1_y.replace(-np.inf,0,inplace=True)

Test2_x.fillna(0,inplace=True)
Test2_x.replace(np.inf,0,inplace=True)
Test2_x.replace(-np.inf,0,inplace=True)

Test2_y.fillna(0,inplace=True)
Test2_y.replace(np.inf,0,inplace=True)
Test2_y.replace(-np.inf,0,inplace=True)

In [16]:
Test1_X = Test1_x.values.reshape((Test1_x.shape[0], 1, Test1_x.shape[1]))
Test1_Y = Test1_y.values
Test2_X = Test2_x.values.reshape((Test2_x.shape[0], 1, Test2_x.shape[1]))
Test2_Y = Test2_y.values
print(Test1_X.shape,Test1_Y.shape,Test2_X.shape,Test2_Y.shape)

(1000000, 1, 1494) (1000000,) (1000000, 1, 1494) (1000000,)


In [17]:
val_pred = model.predict(val_X, verbose=0)
y_pred1 = model.predict(Test1_X, verbose=0)
y_pred2 = model.predict(Test2_X, verbose=0)

In [18]:
def checkAcc(predict, y_test):
    temp = np.multiply(predict, y_test)
    res = [i for i in temp if i > 0]
    acc = len(res)/len(predict)
    print("direction prediction accuracy", acc)
    return acc

In [19]:
val_pred = val_pred.ravel()
checkAcc(val_pred, val_Y)

direction prediction accuracy 0.517051


0.517051

In [20]:
y_pred1 = y_pred1.ravel()
checkAcc(y_pred1, Test1_Y)

direction prediction accuracy 0.51961


0.51961

In [21]:
y_pred2 = y_pred2.ravel()
checkAcc(y_pred2, Test2_Y)

direction prediction accuracy 0.524384


0.524384

In [1]:
from itertools import permutations
from collections import Counter

In [15]:
n = 10
s= str(n)
a =list( permutations(s))
a[0]
int(''.join(a[0]))
list(permutations(str(123)))

[('1', '2', '3'),
 ('1', '3', '2'),
 ('2', '1', '3'),
 ('2', '3', '1'),
 ('3', '1', '2'),
 ('3', '2', '1')]

In [13]:
def solve(n):
    l = len(str(n))
    if l <=3:
        if n%8 == 0:
            return "YES" 
    else:  
        for i in permutations( str(n),3):
            num = int(''.join(i))
            if num %8 == 0:
                return "YES"
    return "NO"

In [ ]:
[line[i:i+3] for i in range(0, len(line)-3)]

In [14]:
n = 51003309490753470524125270008606806367328593031305727464566556358

In [5]:
Counter(str(n))

Counter({'0': 12,
         '1': 3,
         '2': 5,
         '3': 9,
         '4': 5,
         '5': 10,
         '6': 8,
         '7': 6,
         '8': 4,
         '9': 3})

In [15]:
solve(n)

'YES'

In [9]:
len(a)

262080